In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# === Input ===
image_path = input("Enter full path to mercury image: ")
known_total_temp = 100  # Set the thermometer's max temp range

# === Read image ===
img = cv2.imread(image_path)
if img is None:
    print("Error: Image not found.")
    exit()

# === HSV + brightness threshold ===
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v_channel = hsv[:, :, 2]
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

# Red mask range
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Combine brightness + red masks
final_mask = cv2.bitwise_and(red_mask, thresh_mask)

# Detect red mercury region
coords = cv2.findNonZero(final_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])
    mercury_width = max_x - min_x + 1

    # 🧠 Dynamically use actual thermometer width (e.g., 100px = 100°C)
    scale_width = img.shape[1]  # Or use max_x + 1 if full mercury is visible
    temp_per_pixel = known_total_temp / scale_width
    measured_temp = mercury_width * temp_per_pixel

    # Bounding box visualization
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title("Detected Mercury Region")
    plt.axis('off')
    plt.show()

    # Final Output
    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

else:
    print("No red mercury region detected.")
